In [1]:
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
import tensorflow as tf
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Dropout, GaussianNoise, Activation
from keras.optimizers import Adam, SGD
from keras.utils import to_categorical 
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, LabelBinarizer, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix

Using TensorFlow backend.


In [2]:
def data():
    def process_attributes(df, train, val, test):
        cs = MinMaxScaler()
        trainContinuous = cs.fit_transform(train.select_dtypes(np.number))
        valContinuous = cs.transform(val.select_dtypes(np.number))
        testContinuous = cs.transform(test.select_dtypes(np.number))
        #zipBinarizer = LabelBinarizer().fit(df["Cat"])
        #trainCategorical = zipBinarizer.transform(train["Cat"])
        #valCategorical = zipBinarizer.transform(val["Cat"])
        #testCategorical = zipBinarizer.transform(test["Cat"])
        # construct our training and testing data points by concatenating
        # the categorical features with the continuous features
        #trainX = np.hstack([trainCategorical, trainContinuous])
        #testX = np.hstack([testCategorical, testContinuous])
        trainX=trainContinuous
        valX=valContinuous
        testX=testContinuous
        # return the concatenated training and testing data
        return (trainX, valX, testX)
    
    df = pd.read_excel('Colorectal Generated Data_New.xlsx')
    df.set_index('Name',inplace=True)

    train_inputs = pd.read_excel('Train_val_test_split.xlsx', sheet_name='X_train')
    train_inputs.set_index('Name',inplace=True)

    train_outputs = pd.read_excel('Train_val_test_split.xlsx', sheet_name='Y_train')
    train_outputs.set_index('Name',inplace=True)

    val_inputs = pd.read_excel('Train_val_test_split.xlsx', sheet_name='X_val')
    val_inputs.set_index('Name',inplace=True)

    val_outputs = pd.read_excel('Train_val_test_split.xlsx', sheet_name='Y_val')
    val_outputs.set_index('Name',inplace=True)

    test_inputs = pd.read_excel('Train_val_test_split.xlsx', sheet_name='X_test')
    test_inputs.set_index('Name',inplace=True)

    test_outputs = pd.read_excel('Train_val_test_split.xlsx', sheet_name='Y_test')
    test_outputs.set_index('Name',inplace=True)
    
    (X_train, X_val, X_test) = process_attributes(df, train_inputs, val_inputs, test_inputs)
    Y_train = to_categorical(train_outputs)
    Y_val = to_categorical(val_outputs)
    Y_test = to_categorical(test_outputs)
    return X_train, Y_train, X_val, Y_val, X_test, Y_test

In [3]:
def model(X_train, Y_train, X_val, Y_val, X_test, Y_test):
    model=Sequential()
    model.add(Dense({{choice([8,16, 24, 32])}}))
    model.add(Activation({{choice(['relu', 'sigmoid'])}}))
    
    model.add(Dropout({{uniform(0, 0.5)}}))
    
    model.add(Dense({{choice([8,16, 24, 32])}}))
    model.add(Activation({{choice(['relu', 'sigmoid'])}}))
    
    model.add(Dropout({{uniform(0, 0.5)}}))
    
    conditional = {{choice(['two', 'three'])}}
    if conditional == 'three':
        model.add(Dense({{choice([8,16, 24, 32])}}))
        model.add(Activation({{choice(['relu', 'sigmoid'])}}))
        model.add(Dropout({{uniform(0, 0.5)}}))
    
    model.add(Dense(2))
    model.add(Activation('softmax'))
    
    optim=tf.keras.optimizers.Adam()
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer=optim)
    
    model.fit(X_train, Y_train,
              batch_size={{choice([32,64])}},
              epochs=50,
              verbose=2,
              validation_data=(X_val, Y_val))
    score, acc = model.evaluate(X_val, Y_val, verbose=0)
    print('Test score:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [4]:
X_train, Y_train, X_val, Y_val, X_test, Y_test = data()

In [5]:
best_run, best_model = optim.minimize(model=model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=500,
                                      trials=Trials(),
                                      notebook_name='Optimisation ANN kfold')

>>> Imports:
#coding=utf-8

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import Dense, LSTM, SimpleRNN, Dropout, GaussianNoise, Activation
except:
    pass

try:
    from keras.optimizers import Adam, SGD
except:
    pass

try:
    from keras.utils import to_categorical
except:
    pass

try:
    from matplotlib import pyplot as plt
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold
except:
    pass

try:
    from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, LabelBinarizer, MinMaxScaler
except:
    pass

t

Epoch 11/50                                                                                                            
 - 0s - loss: 0.7412 - accuracy: 0.4776 - val_loss: 0.6907 - val_accuracy: 0.6455                                      

Epoch 12/50                                                                                                            
 - 0s - loss: 0.7188 - accuracy: 0.5037 - val_loss: 0.6906 - val_accuracy: 0.5306                                      

Epoch 13/50                                                                                                            
 - 0s - loss: 0.7190 - accuracy: 0.4996 - val_loss: 0.6910 - val_accuracy: 0.4988                                      

Epoch 14/50                                                                                                            
 - 0s - loss: 0.7006 - accuracy: 0.5355 - val_loss: 0.6904 - val_accuracy: 0.5110                                      

Epoch 15/50                         

 - 0s - loss: 0.6853 - accuracy: 0.5460 - val_loss: 0.6830 - val_accuracy: 0.7653                                      

Epoch 45/50                                                                                                            
 - 0s - loss: 0.6900 - accuracy: 0.5306 - val_loss: 0.6826 - val_accuracy: 0.7139                                      

Epoch 46/50                                                                                                            
 - 0s - loss: 0.6870 - accuracy: 0.5526 - val_loss: 0.6818 - val_accuracy: 0.7237                                      

Epoch 47/50                                                                                                            
 - 0s - loss: 0.6853 - accuracy: 0.5485 - val_loss: 0.6811 - val_accuracy: 0.7555                                      

Epoch 48/50                                                                                                            
 - 0s - loss: 0.6875 - accuracy: 0.5

job exception: Cannot clone object '<keras.engine.sequential.Sequential object at 0x000001ABE966E240>' (type <class 'keras.engine.sequential.Sequential'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.


  0%|                                                                          | 0/500 [00:06<?, ?trial/s, best loss=?]


TypeError: Cannot clone object '<keras.engine.sequential.Sequential object at 0x000001ABE966E240>' (type <class 'keras.engine.sequential.Sequential'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.

In [ ]:
print(best_run)

In [ ]:
#Calculate the labels for the test set
predictions = best_model.predict(X_test)


In [ ]:
print(confusion_matrix(np.argmax(Y_test, axis=1), np.argmax(predictions, axis=1)))
print(classification_report(np.argmax(Y_test, axis=1), np.argmax(predictions, axis=1)))